In [ ]:
# KEEP THIS ONE TO SEE ORIGINAL COLUMN NAMES FOR DOCUMENTATION BEFORE RENAMING & ALSO FOR REFERENCES

#VKL_NUMMER = Accident_ID
#AP3_CODE = Accident_Outcome                 ### Reference List: Aflopen3 i
#ANTL_PTJ = Number_Involvers
#AOL_ID = Type_Accident                      ### Reference List: AardOngevallen (Earth Accidents)

#NIVEAUKOP = Road_Level                      ### E=Accident exactly linked to BN, K=K = Accident linked at intersection level,S = Accident linked at street level,G = Accident linked to municipal level
#WSE_ID = Road_Situation                     ### Reference List: wegsituaties (Road Situations)
#WVL_ID = Road_Lightning                     ### Reference List: Wegverlichtingen (Road Lighting)
#WDK_ID = Road_Surface                       ### Reference list: Wegdekken (Road surfaces)
#WGD_CODE_1 = Road_Situation_2               ### D = Droog, R = Regen, M = Mist, S = Sneeuw/Hagel, H = Harde windstoten, O = Onbekend



#IND_ALC = Alcohol_Indication                ### J : Yes , N: No
#MAXSNELHD = Allowed_Max_Speed 
#LGD_ID = Lightning_Conditions_Accident      ### Reference List: Lichtgesteldheden (Lighting Conditions)

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [11]:
df_accidents = pd.read_csv('data/accidents.txt', sep=',', error_bad_lines=False, index_col=False, encoding= 'unicode_escape',dtype='unicode')
df_coordinates = pd.read_csv('data/puntlocaties.txt', sep=',', error_bad_lines=False, index_col=False, encoding= 'unicode_escape',dtype='unicode')
df_objects = pd.read_csv('data/objecttypes.txt', sep=',', error_bad_lines=False, index_col=False, encoding= 'unicode_escape',dtype='unicode')

In [12]:
df_accidents.head()
df_coordinates.head()

,FK_VELD5,X_COORD,Y_COORD
0,WVK0140258021,70074,429373.5
1,WVK0140258026,70094.27,429454.328
2,WVK0140258041,70064.5,429191.5
3,WVK0139257066,69555.68,428620.852
4,WVK0139257087,69680,428777.5


In [13]:
# DROPPING UNNCESSARY COLUMNS & RENAMING WHAT'S LEFT

#df.columns

drop_columns = ['REGNUMMER','PVOPGEM', 'DATUM_VKL', 'DAG_CODE',
       'MND_NUMMER','JAAR_VKL', 'TIJDSTIP', 'UUR', 'DDL_ID',
       'AP4_CODE', 'AP5_CODE', 'ANTL_SLA', 'ANTL_DOD', 'ANTL_GZH', 'ANTL_SEH',
       'ANTL_GOV', 'ANTL_TDT', 'MNE_CODE','WSE_AN', 'BEBKOM','WVL_ID', 'WVG_ID', 'WVG_AN','WDK_AN',
       'ZAD_ID','WGD_CODE_2','BZD_ID_VM1', 'BZD_ID_VM2', 'BZD_ID_VM3', 'BZD_VM_AN', 'BZD_ID_IF1',
       'BZD_ID_IF2', 'BZD_ID_IF3', 'BZD_IF_AN', 'BZD_ID_TA1', 'BZD_ID_TA2',
       'BZD_ID_TA3', 'BZD_TA_AN', 'JTE_ID', 'WVK_ID', 'HECTOMETER',
       'HUISNUMMER','PLT_NAAM', 'DIENSTCODE', 'DIENSTNAAM', 
       'DAGTYPE', 'WEEKNR']

df_accidents = df_accidents.drop(drop_columns,axis=1)


df_accidents.columns=['Accident_ID', 'Accident_Outcome', 'Number_Involvers', 'Type_Accident',
       'Road_Level', 'Road_Situation', 'Allowed_Max_Speed', 'Road_Surface',
       'Lightning_Conditions_Accident', 'Road_Situation_2',"ForeignKey_Locations",
        'City_ID','City_Name','Province_Code', 'Province_Name',
        'Regions','District_Code', 'District_Name', 'Alcohol_Indication']

In [14]:
# MERGING ACCIDENT DF AND COORDINATES DF 

big_df = pd.merge(left=df_accidents, right=df_coordinates, left_on='ForeignKey_Locations',
                  right_on='FK_VELD5', how='left')
big_df["ForeignKey_Locations"].isnull().sum()
big_df=big_df.drop(columns="FK_VELD5")
big_df.head()

,Accident_ID,Accident_Outcome,Number_Involvers,Type_Accident,Road_Level,Road_Situation,Allowed_Max_Speed,Road_Surface,Lightning_Conditions_Accident,Road_Situation_2,...,City_ID,City_Name,Province_Code,Province_Name,Regions,District_Code,District_Name,Alcohol_Indication,X_COORD,Y_COORD
0,20190011333,UMS,2,0,E,1,60,3,3,D,...,85,Ooststellingwerf,FR,Friesland,NaN,NaN,NaN,NaN,214779.378,561175.131
1,20190011334,LET,2,4,S,1,80,3,1,D,...,1681,Borger-Odoorn,DR,Drenthe,NaN,NaN,NaN,NaN,260688.242,543011.462
2,20190011335,UMS,1,0,G,1,30,1,2,S,...,114,Emmen,DR,Drenthe,NaN,100,NN District Oost,NaN,254208.136,527397.527
3,20190011336,LET,3,8,E,2,80,3,2,D,...,1701,Westerveld,DR,Drenthe,NaN,NaN,NaN,NaN,210864.796,532743.692
4,20190011337,UMS,2,8,E,1,130,2,3,R,...,1969,Westerkwartier,GR,Groningen,NaN,100,NN District Oost,NaN,214114.487,574493.475


In [15]:
# CONVERTING CATEGORIZED DATA INTO ITS' REFERENCES

big_df["Accident_Outcome"].replace({"UMS":"Only_material_dmg","LET":"Injured","DOD":"Deadly"}, inplace=True)

big_df["Type_Accident"].replace({"0": "Unkown", "1": "Pedestrian","2":"Parked_Vehicle","3":"Animal","4":"Solid_Object",
                   "5":"Loose_Object","6":"Frontal","7":"Flank","8":"Head/tail","9":"One-sided"}, inplace=True)
   
big_df["Road_Level"].replace({"E": "Accident exactly linked to BN", "K": "Accident linked at intersection level",
                        "S":"Accident linked at street level","G":"Accident linked to municipal level"}, inplace=True)

big_df["Road_Situation"].replace({"1": "Straight_Road", "2":"Bend","3":"Roundabout","4":"Crossroads_3_branches",
                              "5":"Crossroad_4_branches"}, inplace=True)

big_df["Road_Surface"].replace({"1": "Dry", "2": "Wet","3":"Snow"}, inplace=True)

big_df["Road_Situation_2"].replace({"D":"Dry","R":"Rain","M":"Mist","S":"Snow","H":"Strong_Wind","O":"Unkown"}, inplace=True)

big_df["Lightning_Conditions_Accident"].replace({"1": "Daylight", "2": "Darkness","3":"Dusk"}, inplace=True)

big_df.head()

,Accident_ID,Accident_Outcome,Number_Involvers,Type_Accident,Road_Level,Road_Situation,Allowed_Max_Speed,Road_Surface,Lightning_Conditions_Accident,Road_Situation_2,...,City_ID,City_Name,Province_Code,Province_Name,Regions,District_Code,District_Name,Alcohol_Indication,X_COORD,Y_COORD
0,20190011333,Only_material_dmg,2,Unkown,Accident exactly linked to BN,Straight_Road,60,Snow,Dusk,Dry,...,85,Ooststellingwerf,FR,Friesland,NaN,NaN,NaN,NaN,214779.378,561175.131
1,20190011334,Injured,2,Solid_Object,Accident linked at street level,Straight_Road,80,Snow,Daylight,Dry,...,1681,Borger-Odoorn,DR,Drenthe,NaN,NaN,NaN,NaN,260688.242,543011.462
2,20190011335,Only_material_dmg,1,Unkown,Accident linked to municipal level,Straight_Road,30,Dry,Darkness,Snow,...,114,Emmen,DR,Drenthe,NaN,100,NN District Oost,NaN,254208.136,527397.527
3,20190011336,Injured,3,Head/tail,Accident exactly linked to BN,Bend,80,Snow,Darkness,Dry,...,1701,Westerveld,DR,Drenthe,NaN,NaN,NaN,NaN,210864.796,532743.692
4,20190011337,Only_material_dmg,2,Head/tail,Accident exactly linked to BN,Straight_Road,130,Wet,Dusk,Rain,...,1969,Westerkwartier,GR,Groningen,NaN,100,NN District Oost,NaN,214114.487,574493.475


In [16]:
#134617 rows × 21 columns ~2,826,957 records
big_df.isnull().sum()

Accident_ID                           0
Accident_Outcome                      0
Number_Involvers                      0
Type_Accident                         0
Road_Level                            0
Road_Situation                    51768
Allowed_Max_Speed                 31410
Road_Surface                      50929
Lightning_Conditions_Accident         3
Road_Situation_2                  52080
ForeignKey_Locations                  0
City_ID                               0
City_Name                             0
Province_Code                         0
Province_Name                         0
Regions                           20271
District_Code                     90081
District_Name                     90081
Alcohol_Indication               132120
X_COORD                               0
Y_COORD                               0
dtype: int64

In [17]:
# TO WORK ON AMSTERDAM DATA   (AMSTERDAM CITY ID 363)
big_df_amsterdam = big_df[big_df["City_Name"] =="Amsterdam"]
big_df_amsterdam.head()

,Accident_ID,Accident_Outcome,Number_Involvers,Type_Accident,Road_Level,Road_Situation,Allowed_Max_Speed,Road_Surface,Lightning_Conditions_Accident,Road_Situation_2,...,City_ID,City_Name,Province_Code,Province_Name,Regions,District_Code,District_Name,Alcohol_Indication,X_COORD,Y_COORD
1464,20190014869,Only_material_dmg,1,Head/tail,Accident exactly linked to BN,Straight_Road,70,Wet,Daylight,Rain,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,186,WNN District Zuid,NaN,124307.651,491186.807
1489,20190008875,Only_material_dmg,1,Pedestrian,Accident exactly linked to BN,Straight_Road,50,Dry,Daylight,Dry,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,121282.32,486867.997
1490,20190008876,Injured,2,Unkown,Accident exactly linked to BN,Crossroad_4_branches,50,Dry,Daylight,Dry,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,120053.286,487229.077
1510,20190009185,Only_material_dmg,2,Solid_Object,Accident exactly linked to BN,Straight_Road,50,Dry,Daylight,Dry,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,123201.765,486125.974
1599,20190010442,Injured,2,Flank,Accident exactly linked to BN,Crossroad_4_branches,50,Wet,Daylight,Rain,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,119473,484372


In [20]:
big_df_amsterdam

,Accident_ID,Accident_Outcome,Number_Involvers,Type_Accident,Road_Level,Road_Situation,Allowed_Max_Speed,Road_Surface,Lightning_Conditions_Accident,Road_Situation_2,...,City_ID,City_Name,Province_Code,Province_Name,Regions,District_Code,District_Name,Alcohol_Indication,X_COORD,Y_COORD
1464,20190014869,Only_material_dmg,1,Head/tail,Accident exactly linked to BN,Straight_Road,70,Wet,Daylight,Rain,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,186,WNN District Zuid,NaN,124307.651,491186.807
1489,20190008875,Only_material_dmg,1,Pedestrian,Accident exactly linked to BN,Straight_Road,50,Dry,Daylight,Dry,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,121282.32,486867.997
1490,20190008876,Injured,2,Unkown,Accident exactly linked to BN,Crossroad_4_branches,50,Dry,Daylight,Dry,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,120053.286,487229.077
1510,20190009185,Only_material_dmg,2,Solid_Object,Accident exactly linked to BN,Straight_Road,50,Dry,Daylight,Dry,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,123201.765,486125.974
1599,20190010442,Injured,2,Flank,Accident exactly linked to BN,Crossroad_4_branches,50,Wet,Daylight,Rain,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,119473,484372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134387,20190078786,Injured,2,Loose_Object,Accident linked at intersection level,Crossroad_4_branches,30,Dry,Daylight,Dry,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,123993,487230
134388,20190078787,Only_material_dmg,3,Solid_Object,Accident exactly linked to BN,Straight_Road,50,Dry,Daylight,Dry,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,112937.927,485277.83
134389,20190078788,Only_material_dmg,2,Flank,Accident linked at intersection level,Crossroad_4_branches,50,Dry,Darkness,Dry,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,118905,484400
134440,20190180563,Only_material_dmg,2,Unkown,Accident exactly linked to BN,NaN,NaN,NaN,Darkness,NaN,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,186,WNN District Zuid,NaN,118697.869,490922.216


In [19]:
# 7085 rows × 21 columns ~ 150.000 records
big_df_amsterdam.isnull().sum()

# So out of 7.000 cells, we have approximately 2.000 missing for some of them. Lower part is not
# important since they are nonly disctrict related. We already choose the city.

# Still 5.000 records for analyzing is good I believe..

Accident_ID                         0
Accident_Outcome                    0
Number_Involvers                    0
Type_Accident                       0
Road_Level                          0
Road_Situation                   2482
Allowed_Max_Speed                1448
Road_Surface                     2427
Lightning_Conditions_Accident       0
Road_Situation_2                 2541
ForeignKey_Locations                0
City_ID                             0
City_Name                           0
Province_Code                       0
Province_Name                       0
Regions                             0
District_Code                    4661
District_Name                    4661
Alcohol_Indication               6969
X_COORD                             0
Y_COORD                             0
dtype: int64

In [21]:
# ONE HOT DECODING FOR EACH COLUMN TO CHECK CORRELATIONS
    
df_onehot = big_df_amsterdam.copy()

# Accident Outcome Onehot Encoding
df_onehot_accident_outcome = pd.get_dummies(df_onehot, columns=['Accident_Outcome'], prefix = ['Accident_Outcome'])
df_onehot_accident_outcome.head()
#Type of Accident
df_onehot_type_accident = pd.get_dummies(df_onehot, columns=['Type_Accident'], prefix = ['Type_Accident'])
df_onehot_type_accident.head()
# Road Level Onehot Encoding
df_onehot_road_level = pd.get_dummies(df_onehot, columns=['Road_Level'], prefix = ['Road_Level'])
df_onehot_road_level.head()
# Road Situation Onehot Encoding
df_onehot_road_situation = pd.get_dummies(df_onehot, columns=['Road_Situation'], prefix = ['Road_Situation'])
df_onehot_road_situation.head()
# Road Surface Onethot Encoding
df_onehot_road_surface = pd.get_dummies(df_onehot, columns=['Road_Surface'], prefix = ['Road_Surface'])
df_onehot_road_surface.head()
# Lightning Conditions Encoding
df_onehot_lightning_conditions_accident = pd.get_dummies(df_onehot, columns=['Lightning_Conditions_Accident'], prefix = ['Lightning_Conditions_Accident'])
df_onehot_lightning_conditions_accident.head()
# Road Situatuon 2 Onehot Encoding
df_onehot_road_situation_2 = pd.get_dummies(df_onehot, columns=['Road_Situation_2'], prefix = ['Road_Situation_2'])
df_onehot_road_situation_2.head()

,Accident_ID,Accident_Outcome,Number_Involvers,Type_Accident,Road_Level,Road_Situation,Allowed_Max_Speed,Road_Surface,Lightning_Conditions_Accident,ForeignKey_Locations,...,District_Code,District_Name,Alcohol_Indication,X_COORD,Y_COORD,Road_Situation_2_Dry,Road_Situation_2_Mist,Road_Situation_2_Rain,Road_Situation_2_Snow,Road_Situation_2_Strong_Wind
1464,20190014869,Only_material_dmg,1,Head/tail,Accident exactly linked to BN,Straight_Road,70,Wet,Daylight,HTT02483820250040,...,186,WNN District Zuid,NaN,124307.651,491186.807,0,0,1,0,0
1489,20190008875,Only_material_dmg,1,Pedestrian,Accident exactly linked to BN,Straight_Road,50,Dry,Daylight,WVK0242373163,...,NaN,NaN,NaN,121282.32,486867.997,1,0,0,0,0
1490,20190008876,Injured,2,Unkown,Accident exactly linked to BN,Crossroad_4_branches,50,Dry,Daylight,JTE0240374124,...,NaN,NaN,NaN,120053.286,487229.077,1,0,0,0,0
1510,20190009185,Only_material_dmg,2,Solid_Object,Accident exactly linked to BN,Straight_Road,50,Dry,Daylight,WVK0246372051,...,NaN,NaN,NaN,123201.765,486125.974,1,0,0,0,0
1599,20190010442,Injured,2,Flank,Accident exactly linked to BN,Crossroad_4_branches,50,Wet,Daylight,JTE0238368128,...,NaN,NaN,NaN,119473,484372,0,0,1,0,0


In [22]:
# TO GET NUMERIC VALUES ONLY TO ANALYZE - TOGETHER WITH ENCODED COLUMNS / ORIGINAL COLUMNS DROPPED

dicto1 = {1:("Accident_Outcome_Deadly","Accident_Outcome_Injured","Accident_Outcome_Only_material_dmg"),
          2:(df_onehot_accident_outcome),
          3:("Type_Accident_Unkown","Type_Accident_Pedestrian","Type_Accident_Parked_Vehicle","Type_Accident_Animal","Type_Accident_Solid_Object","Type_Accident_Loose_Object","Type_Accident_Frontal","Type_Accident_Flank","Type_Accident_Head/tail","Type_Accident_One-sided"),
          4:(df_onehot_type_accident),
          5:("Road_Level_Accident exactly linked to BN","Road_Level_Accident linked at intersection level","Road_Level_Accident linked at street level","Road_Level_Accident linked to municipal level"),
          6:(df_onehot_road_level),
          7:("Road_Situation_Bend","Road_Situation_Crossroad_4_branches","Road_Situation_Crossroads_3_branches","Road_Situation_Roundabout","Road_Situation_Straight_Road"),
          8:(df_onehot_road_situation),
          9:("Road_Surface_Dry","Road_Surface_Snow","Road_Surface_Wet"),
          10:(df_onehot_road_surface),
          11:("Lightning_Conditions_Accident_Darkness","Lightning_Conditions_Accident_Daylight","Lightning_Conditions_Accident_Dusk"),
          12:(df_onehot_lightning_conditions_accident),
          13:("Road_Situation_2_Dry","Road_Situation_2_Mist","Road_Situation_2_Rain","Road_Situation_2_Snow","Road_Situation_2_Strong_Wind"),
          14:(df_onehot_road_situation_2)}

df_decoded = big_df_amsterdam.copy()  #TO ALL CITIES replace with ==> df_decoded_df=big_df.copy()

o=2
for keys,columns in dicto1.items():
    if keys % 2 != 0:
        for k in range(o,o+1):
            df_decoded[list(columns)] = dicto1[k][list(columns)] 
        o += 2

# NOW DROPPING THE MAIN COLUMNS SINCE WE HAVE THEIR ENCODED VERSION
df.columns=['Accident_ID', 'Accident_Outcome', 'Number_Involvers', 'Type_Accident',
       'Road_Level', 'Road_Situation', 'Allowed_Max_Speed', 'Road_Surface',
       'Lightning_Conditions_Accident', 'Road_Situation_2',"ForeignKey_Locations",
        'City_ID','City_Name','Province_Code', 'Province_Name',
        'Regions','District_Code', 'District_Name', 'Alcohol_Indication']

single_columns = ["Accident_Outcome","Type_Accident","Road_Level","Road_Situation","Road_Surface","Lightning_Conditions_Accident","Road_Situation_2"]
string_columns = ["Accident_ID","ForeignKey_Locations","City_ID","City_Name","Province_Code","Province_Name","Regions","District_Code","District_Name","Alcohol_Indication","X_COORD","Y_COORD"]

df_decoded = df_decoded.drop(columns=single_columns)
df_decoded = df_decoded.drop(columns=string_columns)
df_decoded.head()


,Number_Involvers,Allowed_Max_Speed,Accident_Outcome_Deadly,Accident_Outcome_Injured,Accident_Outcome_Only_material_dmg,Type_Accident_Unkown,Type_Accident_Pedestrian,Type_Accident_Parked_Vehicle,Type_Accident_Animal,Type_Accident_Solid_Object,...,Road_Surface_Snow,Road_Surface_Wet,Lightning_Conditions_Accident_Darkness,Lightning_Conditions_Accident_Daylight,Lightning_Conditions_Accident_Dusk,Road_Situation_2_Dry,Road_Situation_2_Mist,Road_Situation_2_Rain,Road_Situation_2_Snow,Road_Situation_2_Strong_Wind
1464,1,70,0,0,1,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
1489,1,50,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1490,2,50,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1510,2,50,0,0,1,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
1599,2,50,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0


In [23]:
big_df_amsterdam.head()

,Accident_ID,Accident_Outcome,Number_Involvers,Type_Accident,Road_Level,Road_Situation,Allowed_Max_Speed,Road_Surface,Lightning_Conditions_Accident,Road_Situation_2,...,City_ID,City_Name,Province_Code,Province_Name,Regions,District_Code,District_Name,Alcohol_Indication,X_COORD,Y_COORD
1464,20190014869,Only_material_dmg,1,Head/tail,Accident exactly linked to BN,Straight_Road,70,Wet,Daylight,Rain,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,186,WNN District Zuid,NaN,124307.651,491186.807
1489,20190008875,Only_material_dmg,1,Pedestrian,Accident exactly linked to BN,Straight_Road,50,Dry,Daylight,Dry,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,121282.32,486867.997
1490,20190008876,Injured,2,Unkown,Accident exactly linked to BN,Crossroad_4_branches,50,Dry,Daylight,Dry,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,120053.286,487229.077
1510,20190009185,Only_material_dmg,2,Solid_Object,Accident exactly linked to BN,Straight_Road,50,Dry,Daylight,Dry,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,123201.765,486125.974
1599,20190010442,Injured,2,Flank,Accident exactly linked to BN,Crossroad_4_branches,50,Wet,Daylight,Rain,...,363,Amsterdam,NH,Noord-Holland,REGIONAAL ORGAAN AMSTERDAM,NaN,NaN,NaN,119473,484372


In [ ]:
#CHECK CORRELATIONS BETWEEN COLUMNS TO HAVE LOGICAL ANSWERS



## Questions

1) Accident outcome - road situation_2 (If wet, death ratio higher?) -check when it's wet, when it's dry
2) Group by city and sum accident ID's to have an idea about number of accidents in each city 
3) Accident outcome - road lighthning (If day time, accidents more predictable, less death ratio maybe)
4) Accident outcome - alcohol indication (if yes more deaths)
5) Type of accident happened most in Amsterdam (Refer AardOngevallen.txt and merge the dataframe for demonstration??)

